# 账户、地址与钱包

单击顶部栏上的`🚀` -&gt; `Binder`在线运行此示例！

## 准备

本部分将准备`web3`对象与私钥`private_key`，提供后文代码所需的变量。

In [2]:
# 领取 1000 CFX 到私钥对应的账户
from pprint import pprint
from conflux_web3 import Web3

w3_ = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))

acct = w3_.account.create()

w3_.cfx.default_account = acct

faucet = w3_.cfx.contract(name="Faucet")
tx_receipt = faucet.functions.claimCfx().transact().executed()

# 新建w3对象以便演示API的用法
w3 = Web3(Web3.HTTPProvider("https://test.confluxrpc.com"))
private_key: str = acct.key.hex()

## 账户与地址

在 web3 中，控制着一个帐户意味着知道一个私钥。虽然私钥应该保密，但***地址***不必。地址是从私钥单向派生的字符串，用于标识帐户。不同区块链常常会使用不同方式编码账户地址，例如比特币与以太坊的地址编码方式就存在区别。在 Conflux 中，地址以[CIP-37](https://github.com/Conflux-Chain/CIPs/blob/master/CIPs/cip-37.md)定义的 base32 格式编码。


### `LocalAccount`对象

`w3.account`是一个工厂对象，用于生成用来签署交易的`LocalAccount`对象（例如`random_account` ）。

**注意：手动签署交易相当繁琐。请参阅[钱包](#wallet)部分了解如何使用钱包签名和发送交易。或者您可以参考[从头构建交易](./11-construct_transaction_from_scratch.ipynb)了解如何正确手动签名交易。**

> 更多文档： `w3.account`是一个`cfx_account.Account`对象，其继承自`eth_account.Account` ，大部分api与[eth_account](https://eth-account.readthedocs.io/en/stable/eth_account.html)一致。

In [3]:
random_account = w3.account.from_key(private_key)
print(f"account address: {random_account.address}")
print(f"account private key: {private_key}")

transaction = {
    'to': w3.address.zero_address(),
    'nonce': 1,
    'value': 1,
    'gas': 21000,
    'gasPrice': 10**9,
    'storageLimit': 0,
    'epochHeight': 100,
    'chainId': 1
}
print(f"signed raw tx: {random_account.sign_transaction(transaction).rawTransaction.hex()}")

account address: cfxtest:aanywwykvp5nv9atnh7p287s2r48wcgf9j3xjce7kj
account private key: 0x11d748040e0c1da3d4342a25e98ac4f9463b9b11491a04201834914c35b5670c
signed raw tx: 0xf867e301843b9aca00825208940000000000000000000000000000000000000000018064018001a0eedd9a79b05d1a42f75e972e2ef2b6536a49de81a11f26d9095c146ce95ee314a00ed7bd5f8cef819175525988970c4eb7739755b7138afdd4d5b671acb55c9b49


### Conflux 地址

在 Conflux 中，地址按照[CIP-37](https://github.com/Conflux-Chain/CIPs/blob/master/CIPs/cip-37.md)以 base32 格式编码。你可以简单地从地址字面推断出该地址属于主网、测试网或其他网络。

In [4]:
# "cfxtest" 代表该地址为测试网地址
assert random_account.address.startswith("cfxtest:")

SDK 返回的地址都被类`Base32Address`包裹。该类提供了方便的方法来操作 base32 地址，但你也可以将`Base32Address`对象用作普通的 python `str`对象。[Base32Address 文档](https://conflux-fans.github.io/cfx-address/cfx_address.html#module-cfx_address.address)提供了更详细的代码示例。

In [5]:
addr = random_account.address
print(f"the type of addr: {type(addr)}")
# a Base32Address object is also a `str`
assert isinstance(addr, str)

# encode a base32 address from hex address and network_id
# it is also supported to use `w3.address("cfxtest:aatp533cg7d0agbd87kz48nj1mpnkca8be1rz695j4")`
address = w3.address("0x1ecde7223747601823f7535d7968ba98b4881e09", network_id=1)
print(address)
pprint([
    address.address_type,
    address.network_id,
    address.hex_address,
    address.verbose_address,
    address.abbr,
    address.mapped_evm_space_address,
    address.eth_checksum_address,
])

the type of addr: <class 'cfx_address.address.Base32Address'>
cfxtest:aatp533cg7d0agbd87kz48nj1mpnkca8be1rz695j4
['user',
 1,
 '0x1ecde7223747601823f7535d7968ba98b4881e09',
 'CFXTEST:TYPE.USER:AATP533CG7D0AGBD87KZ48NJ1MPNKCA8BE1RZ695J4',
 'cfxtest:aat...95j4',
 '0x349f086998cF4a0C5a00b853a0E93239D81A97f6',
 '0x1ECdE7223747601823f7535d7968Ba98b4881E09']


## 钱包

我们使用钱包中间件(`w3.wallet`)来帮助我们签名发送交易。如果某未签名交易的`from`账户已添加至`w3.wallet`中，它将对通过`w3.cfx.send_transaction`发送的未签名交易进行签名。

> `wallet`中间件的行为与`web3.py`的`construct_sign_and_send_raw_middleware`基本一致，但提供了更多功能。例如，我们可以使用`w3.wallet.add_account` 、 `w3.wallet.add_accounts` 、 `w3.wallet.pop`来动态添加或删除帐户。 

In [6]:
# wallet是 conflux_web3 的中间件，可向其中添加 LocalAccount
assert w3.wallet is w3.middleware_onion["wallet"]

w3.wallet.add_account(random_account)

assert random_account.address in w3.wallet
w3.cfx.send_transaction({
    "from": random_account.address,
    "to": random_account.address,
    "value": 10**18
}).executed()

AttributeDict({'transactionHash': HexBytes('0x1b53a9e6efa47aa2380ab457b2d7a3a079d14fc362a7bc8830d53e56287f1aa2'),
 'index': 0,
 'blockHash': HexBytes('0x466eeaec283b8a272179e3d5adcec05f7aadd010e28ab740bd18d68da9272f8a'),
 'epochNumber': 99718048,
 'from': 'cfxtest:aanywwykvp5nv9atnh7p287s2r48wcgf9j3xjce7kj',
 'to': 'cfxtest:aanywwykvp5nv9atnh7p287s2r48wcgf9j3xjce7kj',
 'gasUsed': 21000,
 'gasFee': 21000000000000 Drip,
 'contractCreated': None,
 'logs': [],
 'logsBloom': HexBytes('0x00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000'

### 使用`w3.cfx.default_account`将账户添加到钱包

当 `w3.cfx.default_account` 被设置后，若交易的 `from` 字段未指定，则将被自动设置为 `w3.cfx.default_account`。

`w3.cfx.default_account`是一个地址类型的变量，但可以使用`LocalAccount`对象进行设置。在这种情况下，该`LocalAccount`将同时添加到钱包中。

```python
w3.cfx.default_account = random_account
```

相当于

```python
w3.cfx.default_account = random_account.address
w3.wallet.add_account(random_account)
```